In [12]:
from pathlib import Path

# Paths to your files (edit this to your folder)
# data_dir = Path("/../en-ta.txt/")
en_path = "/Users/niteeshkumar/Documents/document_level_nmt/en-ta.txt/pmindia.en-ta.en"
ta_path = "/Users/niteeshkumar/Documents/document_level_nmt/en-ta.txt/pmindia.en-ta.ta"

# 1. Read files
with open(en_path, "r", encoding="utf-8") as f_en:
    en_lines = [line.strip() for line in f_en]

with open(ta_path, "r", encoding="utf-8") as f_ta:
    ta_lines = [line.strip() for line in f_ta]

assert len(en_lines) == len(ta_lines), "Line counts do not match!"

# 2. Build parallel sentence pairs
parallel_pairs = [
    {"en": src, "ta": tgt}
    for src, tgt in zip(en_lines, ta_lines)
    if src and tgt  # drop empty lines
]

print(f"Total pairs: {len(parallel_pairs)}")
print(parallel_pairs[0])


Total pairs: 10577
{'en': 'Prime Minister Narendra Modi met Her Majesty Queen Maxima of the Kingdom of the Netherlands today.', 'ta': 'சுகாதாரம் மற்றும் குடும்பநலத் துறையின் கீழ் உள்ள தன்னாட்சி அமைப்புகளை சீரமைக்கும் நடவடிக்கையில், அமைச்சகங்களுக்கு இடையேயான ஆலோசனைகள் நடைபெறும் மற்றும் இந்த அமைப்புகளுக்கான ஏற்கனவே உள்ள சட்டதிட்டங்கள் மறுஆய்வு செய்யப்படும்.'}


In [13]:
from pathlib import Path

tsv_path = "/Users/niteeshkumar/Documents/document_level_nmt/pmindia.v1.ta-en.tsv" # update this

with open(tsv_path, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        print(repr(line))
        if i == 9:   # first 10 lines
            break


'The rationalization of Autonomous Bodies under Department of Health & Family Welfare will involve inter-ministerial consultations and review of existing bye laws of these bodies.\tசுகாதாரம் மற்றும் குடும்பநலத் துறையின் கீழ் உள்ள தன்னாட்சி அமைப்புகளை சீரமைக்கும் நடவடிக்கையில், அமைச்சகங்களுக்கு இடையேயான ஆலோசனைகள் நடைபெறும் மற்றும் இந்த அமைப்புகளுக்கான ஏற்கனவே உள்ள சட்டதிட்டங்கள் மறுஆய்வு செய்யப்படும்.\n'
'The time frame for implementation is one year,\tஇதனை செயல்படுத்துவதற்கான காலக்கெடு ஓராண்டு.\n'
'Since the DoHFW provides funds to the hospitals, the grants can be given from the Department to the hospital directly.\tமருத்துவமனைகளுக்கான நிதியை சுகாதாரம் மற்றும் குடும்ப நலத் துறை வழங்கி வருவதால், நிதியுதவியையும் மருத்துமனைகளுக்கு துறையே நேரில் வழங்க முடியும்.\n'
'RAN functions can, therefore, be vested in DoHFW.\tதேசிய ஆரோக்கிய நிதி அமைப்பின் செயல்பாடுகள், சுகாதாரம் மற்றும் குடும்பநலத் துறையிடம் இருக்கும்.\n'
'Managing Committee of RAN Society will meet to dissolve the Autonomous Body (A

In [14]:
pairs = []
with open(tsv_path, "r", encoding="utf-8") as f:
    for line in f:
        line = line.rstrip("\n")
        if not line:
            continue

        parts = line.split("\t")
        if len(parts) != 2:
            continue  # skip any abnormal lines

        en, ta = parts
        en, ta = en.strip(), ta.strip()
        if en and ta:
            pairs.append({"en": en, "ta": ta})

print("Total pairs:", len(pairs))
print(pairs[0])

Total pairs: 39526
{'en': 'The rationalization of Autonomous Bodies under Department of Health & Family Welfare will involve inter-ministerial consultations and review of existing bye laws of these bodies.', 'ta': 'சுகாதாரம் மற்றும் குடும்பநலத் துறையின் கீழ் உள்ள தன்னாட்சி அமைப்புகளை சீரமைக்கும் நடவடிக்கையில், அமைச்சகங்களுக்கு இடையேயான ஆலோசனைகள் நடைபெறும் மற்றும் இந்த அமைப்புகளுக்கான ஏற்கனவே உள்ள சட்டதிட்டங்கள் மறுஆய்வு செய்யப்படும்.'}


In [15]:
# !pip install -U sentence-transformers

In [16]:
# from sentence_transformers import SentenceTransformer, util
# import torch

# model = SentenceTransformer("sentence-transformers/LaBSE")

# batch_size = 64
# aligned_pairs = []

# for start in range(0, len(pairs), batch_size):
#     end = start + batch_size
#     en_batch = [p["en"] for p in pairs[start:end]]
#     ta_batch = [p["ta"] for p in pairs[start:end]]

#     en_emb = model.encode(en_batch, convert_to_tensor=True)
#     ta_emb = model.encode(ta_batch, convert_to_tensor=True)

#     cos_scores = util.cos_sim(en_emb, ta_emb).diagonal()  # only batch diag

#     for pair, score in zip(pairs[start:end], cos_scores):
#         if score.item() >= 0.7:
#             aligned_pairs.append({
#                 "en": pair["en"],
#                 "ta": pair["ta"],
#                 "score": float(score),
#             })

# print("Original:", len(pairs), "Aligned:", len(aligned_pairs))


In [17]:
# shape(aligned_pairs)

In [18]:
DOC_LEN = 8  # sentences per pseudo-document

documents = []
for i in range(0, len(pairs), DOC_LEN):
    chunk = pairs[i:i+DOC_LEN]
    if len(chunk) < 2:
        break
    documents.append(chunk)

print("Num documents:", len(documents))
print("First doc len:", len(documents[0]))

Num documents: 4941
First doc len: 8


In [19]:
documents[0]

[{'en': 'The rationalization of Autonomous Bodies under Department of Health & Family Welfare will involve inter-ministerial consultations and review of existing bye laws of these bodies.',
  'ta': 'சுகாதாரம் மற்றும் குடும்பநலத் துறையின் கீழ் உள்ள தன்னாட்சி அமைப்புகளை சீரமைக்கும் நடவடிக்கையில், அமைச்சகங்களுக்கு இடையேயான ஆலோசனைகள் நடைபெறும் மற்றும் இந்த அமைப்புகளுக்கான ஏற்கனவே உள்ள சட்டதிட்டங்கள் மறுஆய்வு செய்யப்படும்.'},
 {'en': 'The time frame for implementation is one year,',
  'ta': 'இதனை செயல்படுத்துவதற்கான காலக்கெடு ஓராண்டு.'},
 {'en': 'Since the DoHFW provides funds to the hospitals, the grants can be given from the Department to the hospital directly.',
  'ta': 'மருத்துவமனைகளுக்கான நிதியை சுகாதாரம் மற்றும் குடும்ப நலத் துறை வழங்கி வருவதால், நிதியுதவியையும் மருத்துமனைகளுக்கு துறையே நேரில் வழங்க முடியும்.'},
 {'en': 'RAN functions can, therefore, be vested in DoHFW.',
  'ta': 'தேசிய ஆரோக்கிய நிதி அமைப்பின் செயல்பாடுகள், சுகாதாரம் மற்றும் குடும்பநலத் துறையிடம் இருக்கும்.'},
 {'en':

In [20]:
len(pairs)

39526

In [21]:
import pandas as pd

df = pd.DataFrame(pairs)
df

,en,ta
0,The rationalization of Autonomous Bodies under...,சுகாதாரம் மற்றும் குடும்பநலத் துறையின் கீழ் உள...
1,"The time frame for implementation is one year,",இதனை செயல்படுத்துவதற்கான காலக்கெடு ஓராண்டு.
2,Since the DoHFW provides funds to the hospital...,மருத்துவமனைகளுக்கான நிதியை சுகாதாரம் மற்றும் க...
3,"RAN functions can, therefore, be vested in DoHFW.","தேசிய ஆரோக்கிய நிதி அமைப்பின் செயல்பாடுகள், சு..."
4,Managing Committee of RAN Society will meet to...,"சங்கங்கள் பதிவுச் சட்டம், 1860-ல் உள்ள வழிமுறை..."
...,...,...
39521,We will also get to see its direct benefit dur...,இரு நாடுகளையும் சேர்ந்த தொழில் துறையினருடன் இன...
39522,I am confident that our dialogue and the agree...,இன்றைய நமது பேச்சுவார்த்தைகள் மற்றும் கையெழுத்...
39523,"With these words, I once again warmly welcome ...","இந்தக் கருத்துகளுடன், பிரதமர் அபே-வையும், அவரு..."
39524,Ijyo De Gozaimas (That’s all for now),Ijyo De Gozaimas (தற்போது இவ்வளவு தான்)


In [22]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import math
# import json
# from torch.utils.data import Dataset, DataLoader
# from collections import Counter
# import numpy as np

# class PositionalEncoding(nn.Module):
#     def __init__(self, d_model, max_len=5000):
#         super().__init__()
#         pe = torch.zeros(max_len, d_model)
#         position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
#         div_term = torch.exp(torch.arange(0, d_model, 2).float() * 
#                            (-math.log(10000.0) / d_model))
#         pe[:, 0::2] = torch.sin(position * div_term)
#         pe[:, 1::2] = torch.cos(position * div_term)
#         pe = pe.unsqueeze(0).transpose(0, 1)
#         self.register_buffer('pe', pe)

#     def forward(self, x):
#         return x + self.pe[:x.size(0), :]

In [23]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

# ---------- Positional Encoding ----------

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)          # [max_len, d_model]
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float()
                             * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)                        # [1, max_len, d_model]
        self.register_buffer("pe", pe)

    def forward(self, x):
        # x: [batch_size, seq_len, d_model]
        x = x + self.pe[:, :x.size(1), :]
        return self.dropout(x)


# ---------- Multi-Head Attention ----------

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_heads, dropout=0.1):
        super().__init__()
        assert d_model % n_heads == 0
        self.d_model = d_model
        self.n_heads = n_heads
        self.d_k = d_model // n_heads

        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

        self.dropout = nn.Dropout(dropout)

    def forward(self, query, key, value, mask=None):
        # query/key/value: [batch, seq_len, d_model]
        batch_size = query.size(0)

        # Linear projections
        Q = self.W_q(query)  # [batch, seq, d_model]
        K = self.W_k(key)
        V = self.W_v(value)

        # Split into heads: [batch, heads, seq, d_k]
        def split_heads(x):
            return x.view(batch_size, -1, self.n_heads, self.d_k).transpose(1, 2)

        Q = split_heads(Q)
        K = split_heads(K)
        V = split_heads(V)

        # Scaled dot-product attention
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        # scores: [batch, heads, seq_q, seq_k]

        if mask is not None:
            # mask: [batch, 1, 1, seq_k] or [batch, 1, seq_q, seq_k]
            scores = scores.masked_fill(mask == 0, float("-inf"))

        attn = F.softmax(scores, dim=-1)
        attn = self.dropout(attn)

        context = torch.matmul(attn, V)  # [batch, heads, seq_q, d_k]

        # Concatenate heads
        context = context.transpose(1, 2).contiguous().view(
            batch_size, -1, self.d_model
        )  # [batch, seq_q, d_model]

        out = self.W_o(context)
        return out


# ---------- Position-wise Feed Forward ----------

class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super().__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.fc2(self.dropout(F.relu(self.fc1(x))))


# ---------- Encoder / Decoder Layers ----------

class EncoderLayer(nn.Module):
    def __init__(self, d_model, n_heads, d_ff, dropout=0.1):
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, n_heads, dropout)
        self.ff = PositionwiseFeedForward(d_model, d_ff, dropout)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, src_mask=None):
        # Self-attention
        attn_out = self.self_attn(x, x, x, src_mask)
        x = self.norm1(x + self.dropout(attn_out))
        # Feed-forward
        ff_out = self.ff(x)
        x = self.norm2(x + self.dropout(ff_out))
        return x


class DecoderLayer(nn.Module):
    def __init__(self, d_model, n_heads, d_ff, dropout=0.1):
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, n_heads, dropout)
        self.cross_attn = MultiHeadAttention(d_model, n_heads, dropout)
        self.ff = PositionwiseFeedForward(d_model, d_ff, dropout)

        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, memory, tgt_mask=None, memory_mask=None):
        # Masked self-attention (decoder)
        self_attn_out = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(self_attn_out))
        # Encoder–decoder attention
        cross_attn_out = self.cross_attn(x, memory, memory, memory_mask)
        x = self.norm2(x + self.dropout(cross_attn_out))
        # Feed-forward
        ff_out = self.ff(x)
        x = self.norm3(x + self.dropout(ff_out))
        return x


# ---------- Encoder / Decoder Stacks ----------

class Encoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, n_heads, d_ff, dropout=0.1, max_len=5000):
        super().__init__()
        self.tok_embed = nn.Embedding(vocab_size, d_model)
        self.pos_embed = PositionalEncoding(d_model, max_len, dropout)
        self.layers = nn.ModuleList(
            [EncoderLayer(d_model, n_heads, d_ff, dropout) for _ in range(N)]
        )

    def forward(self, src, src_mask=None):
        # src: [batch, src_len]
        x = self.tok_embed(src)   # [batch, src_len, d_model]
        x = self.pos_embed(x)
        for layer in self.layers:
            x = layer(x, src_mask)
        return x


class Decoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, n_heads, d_ff, dropout=0.1, max_len=5000):
        super().__init__()
        self.tok_embed = nn.Embedding(vocab_size, d_model)
        self.pos_embed = PositionalEncoding(d_model, max_len, dropout)
        self.layers = nn.ModuleList(
            [DecoderLayer(d_model, n_heads, d_ff, dropout) for _ in range(N)]
        )
        self.fc_out = nn.Linear(d_model, vocab_size)

    def forward(self, tgt, memory, tgt_mask=None, memory_mask=None):
        # tgt: [batch, tgt_len]
        x = self.tok_embed(tgt)
        x = self.pos_embed(x)
        for layer in self.layers:
            x = layer(x, memory, tgt_mask, memory_mask)
        logits = self.fc_out(x)
        return logits


# ---------- Full Transformer ----------

def generate_square_subsequent_mask(sz, device):
    # standard causal mask for decoder
    mask = torch.triu(torch.ones(sz, sz, device=device), diagonal=1).bool()
    return ~mask  # True where allowed, False where masked


class Transformer(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        tgt_vocab_size,
        d_model=512,
        N=6,
        n_heads=8,
        d_ff=2048,
        dropout=0.1,
        max_len=5000,
    ):
        super().__init__()
        self.encoder = Encoder(src_vocab_size, d_model, N, n_heads, d_ff, dropout, max_len)
        self.decoder = Decoder(tgt_vocab_size, d_model, N, n_heads, d_ff, dropout, max_len)

    def make_src_mask(self, src, pad_idx):
        # src: [batch, src_len]
        mask = (src != pad_idx).unsqueeze(1).unsqueeze(2)
        # [batch, 1, 1, src_len]
        return mask

    def make_tgt_mask(self, tgt, pad_idx):
        # tgt: [batch, tgt_len]
        batch_size, tgt_len = tgt.size()
        pad_mask = (tgt != pad_idx).unsqueeze(1).unsqueeze(2)  # [batch, 1, 1, tgt_len]
        subseq_mask = generate_square_subsequent_mask(tgt_len, tgt.device)  # [tgt_len, tgt_len]
        subseq_mask = subseq_mask.unsqueeze(0).unsqueeze(1)  # [1, 1, tgt_len, tgt_len]
        mask = pad_mask & subseq_mask
        return mask

    def forward(self, src, tgt, src_pad_idx, tgt_pad_idx):
        src_mask = self.make_src_mask(src, src_pad_idx)
        tgt_mask = self.make_tgt_mask(tgt, tgt_pad_idx)
        memory = self.encoder(src, src_mask)
        out = self.decoder(tgt, memory, tgt_mask, src_mask)
        return out



# Pseudo-code: adapt to your actual file format.
def parse_line(line):
    # Example input:
    # "enta0The rationalization ...சுகாதாரம் ...1"
    # You likely need a known separator; if not, pre-clean in a script.
    # For illustration, assume tab-separated: id \t en \t ta \t idx
    parts = line.strip().split("\t")
    en = parts[1]
    ta = parts[2]
    return en, ta


In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# After you build vocabularies:
src_vocab_size = len(src_vocab)
tgt_vocab_size = len(tgt_vocab)
pad_idx = 0  # make sure 0 is PAD in both vocabs

model = Transformer(
    src_vocab_size=src_vocab_size,
    tgt_vocab_size=tgt_vocab_size,
    d_model=512,
    N=6,
    n_heads=8,
    d_ff=2048,
    dropout=0.1,
    max_len=256,
).to(device)

criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=1e-4,
    betas=(0.9, 0.98),
    eps=1e-9,
)

for epoch in range(10):
    model.train()
    total_loss = 0.0
    for batch in train_dataloader:
        src, tgt = batch["src"].to(device), batch["tgt"].to(device)
        # Teacher forcing: decoder input is tgt[:, :-1], target labels are tgt[:, 1:]
        tgt_inp = tgt[:, :-1]
        tgt_out = tgt[:, 1:]

        logits = model(src, tgt_inp, pad_idx, pad_idx)  # [batch, tgt_len-1, vocab]
        loss = criterion(
            logits.reshape(-1, tgt_vocab_size),
            tgt_out.reshape(-1)
        )

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}: loss = {total_loss / len(train_dataloader):.4f}")


NameError: name 'src_vocab' is not defined

In [ ]:
#vanila transformer 

In [ ]:
import math
from collections import Counter

import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


# ===============================
# 1. Vocab building & tokenization
# ===============================

PAD_TOKEN = "<pad>"
SOS_TOKEN = "<sos>"
EOS_TOKEN = "<eos>"
UNK_TOKEN = "<unk>"

special_tokens = [PAD_TOKEN, SOS_TOKEN, EOS_TOKEN, UNK_TOKEN]

def simple_tokenize(text: str):
    # Replace with a better tokenizer if you want.
    return text.strip().split()

def build_vocab(texts, min_freq=1):
    counter = Counter()
    for t in texts:
        counter.update(simple_tokenize(str(t)))
    itos = list(special_tokens)
    for tok, freq in counter.items():
        if freq >= min_freq and tok not in itos:
            itos.append(tok)
    stoi = {tok: idx for idx, tok in enumerate(itos)}
    return stoi, itos

# Build vocabularies from dataframe
src_vocab, src_itos = build_vocab(df["en"].tolist(), min_freq=1)
tgt_vocab, tgt_itos = build_vocab(df["ta"].tolist(), min_freq=1)

pad_idx = src_vocab[PAD_TOKEN]
assert pad_idx == tgt_vocab[PAD_TOKEN]

def encode_sentence(text, vocab, max_len=256):
    tokens = [SOS_TOKEN] + simple_tokenize(str(text)) + [EOS_TOKEN]
    ids = [vocab.get(tok, vocab[UNK_TOKEN]) for tok in tokens]
    if len(ids) < max_len:
        ids += [vocab[PAD_TOKEN]] * (max_len - len(ids))
    else:
        ids = ids[:max_len]
    return ids

max_len = 256

src_seqs = [encode_sentence(t, src_vocab, max_len=max_len) for t in df["en"]]
tgt_seqs = [encode_sentence(t, tgt_vocab, max_len=max_len) for t in df["ta"]]

src_tensor = torch.tensor(src_seqs, dtype=torch.long)
tgt_tensor = torch.tensor(tgt_seqs, dtype=torch.long)

# ===============================
# 2. Dataset & DataLoader
# ===============================

class NMTDataset(Dataset):
    def __init__(self, src_tensor, tgt_tensor):
        self.src = src_tensor
        self.tgt = tgt_tensor

    def __len__(self):
        return self.src.size(0)

    def __getitem__(self, idx):
        return {
            "src": self.src[idx],
            "tgt": self.tgt[idx],
        }

dataset = NMTDataset(src_tensor, tgt_tensor)

train_dataloader = DataLoader(
    dataset,
    batch_size=64,
    shuffle=True,
    drop_last=True,
)

# ===============================
# 3. Transformer components
# ===============================

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)          # [max_len, d_model]
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float()
                             * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)                        # [1, max_len, d_model]
        self.register_buffer("pe", pe)

    def forward(self, x):
        # x: [batch_size, seq_len, d_model]
        x = x + self.pe[:, :x.size(1), :]
        return self.dropout(x)


class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_heads, dropout=0.1):
        super().__init__()
        assert d_model % n_heads == 0
        self.d_model = d_model
        self.n_heads = n_heads
        self.d_k = d_model // n_heads

        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

        self.dropout = nn.Dropout(dropout)

    def forward(self, query, key, value, mask=None):
        # query/key/value: [batch, seq_len, d_model]
        batch_size = query.size(0)

        Q = self.W_q(query)
        K = self.W_k(key)
        V = self.W_v(value)

        def split_heads(x):
            return x.view(batch_size, -1, self.n_heads, self.d_k).transpose(1, 2)

        Q = split_heads(Q)
        K = split_heads(K)
        V = split_heads(V)

        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        # scores: [batch, heads, seq_q, seq_k]

        if mask is not None:
            # mask: [batch, 1, 1, seq_k] or [batch, 1, seq_q, seq_k]
            scores = scores.masked_fill(mask == 0, float("-inf"))

        attn = F.softmax(scores, dim=-1)
        attn = self.dropout(attn)

        context = torch.matmul(attn, V)  # [batch, heads, seq_q, d_k]

        context = context.transpose(1, 2).contiguous().view(
            batch_size, -1, self.d_model
        )  # [batch, seq_q, d_model]

        out = self.W_o(context)
        return out


class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super().__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.fc2(self.dropout(F.relu(self.fc1(x))))


class EncoderLayer(nn.Module):
    def __init__(self, d_model, n_heads, d_ff, dropout=0.1):
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, n_heads, dropout)
        self.ff = PositionwiseFeedForward(d_model, d_ff, dropout)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, src_mask=None):
        attn_out = self.self_attn(x, x, x, src_mask)
        x = self.norm1(x + self.dropout(attn_out))
        ff_out = self.ff(x)
        x = self.norm2(x + self.dropout(ff_out))
        return x


class DecoderLayer(nn.Module):
    def __init__(self, d_model, n_heads, d_ff, dropout=0.1):
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, n_heads, dropout)
        self.cross_attn = MultiHeadAttention(d_model, n_heads, dropout)
        self.ff = PositionwiseFeedForward(d_model, d_ff, dropout)

        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, memory, tgt_mask=None, memory_mask=None):
        self_attn_out = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(self_attn_out))
        cross_attn_out = self.cross_attn(x, memory, memory, memory_mask)
        x = self.norm2(x + self.dropout(cross_attn_out))
        ff_out = self.ff(x)
        x = self.norm3(x + self.dropout(ff_out))
        return x


class Encoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, n_heads, d_ff, dropout=0.1, max_len=5000):
        super().__init__()
        self.tok_embed = nn.Embedding(vocab_size, d_model)
        self.pos_embed = PositionalEncoding(d_model, max_len, dropout)
        self.layers = nn.ModuleList(
            [EncoderLayer(d_model, n_heads, d_ff, dropout) for _ in range(N)]
        )

    def forward(self, src, src_mask=None):
        x = self.tok_embed(src)   # [batch, src_len, d_model]
        x = self.pos_embed(x)
        for layer in self.layers:
            x = layer(x, src_mask)
        return x


class Decoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, n_heads, d_ff, dropout=0.1, max_len=5000):
        super().__init__()
        self.tok_embed = nn.Embedding(vocab_size, d_model)
        self.pos_embed = PositionalEncoding(d_model, max_len, dropout)
        self.layers = nn.ModuleList(
            [DecoderLayer(d_model, n_heads, d_ff, dropout) for _ in range(N)]
        )
        self.fc_out = nn.Linear(d_model, vocab_size)

    def forward(self, tgt, memory, tgt_mask=None, memory_mask=None):
        x = self.tok_embed(tgt)
        x = self.pos_embed(x)
        for layer in self.layers:
            x = layer(x, memory, tgt_mask, memory_mask)
        logits = self.fc_out(x)
        return logits


def generate_square_subsequent_mask(sz, device):
    mask = torch.triu(torch.ones(sz, sz, device=device), diagonal=1).bool()
    return ~mask  # True where allowed, False where masked


class Transformer(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        tgt_vocab_size,
        d_model=512,
        N=6,
        n_heads=8,
        d_ff=2048,
        dropout=0.1,
        max_len=5000,
    ):
        super().__init__()
        self.encoder = Encoder(src_vocab_size, d_model, N, n_heads, d_ff, dropout, max_len)
        self.decoder = Decoder(tgt_vocab_size, d_model, N, n_heads, d_ff, dropout, max_len)

    def make_src_mask(self, src, pad_idx):
        mask = (src != pad_idx).unsqueeze(1).unsqueeze(2)
        return mask

    def make_tgt_mask(self, tgt, pad_idx):
        batch_size, tgt_len = tgt.size()
        pad_mask = (tgt != pad_idx).unsqueeze(1).unsqueeze(2)  # [batch, 1, 1, tgt_len]
        subseq_mask = generate_square_subsequent_mask(tgt_len, tgt.device)  # [tgt_len, tgt_len]
        subseq_mask = subseq_mask.unsqueeze(0).unsqueeze(1)  # [1, 1, tgt_len, tgt_len]
        mask = pad_mask & subseq_mask
        return mask

    def forward(self, src, tgt, src_pad_idx, tgt_pad_idx):
        src_mask = self.make_src_mask(src, src_pad_idx)
        tgt_mask = self.make_tgt_mask(tgt, tgt_pad_idx)
        memory = self.encoder(src, src_mask)
        out = self.decoder(tgt, memory, tgt_mask, src_mask)
        return out

# ===============================
# 4. Training setup
# ===============================

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

src_vocab_size = len(src_vocab)
tgt_vocab_size = len(tgt_vocab)

model = Transformer(
    src_vocab_size=src_vocab_size,
    tgt_vocab_size=tgt_vocab_size,
    d_model=512,
    N=6,
    n_heads=8,
    d_ff=2048,
    dropout=0.1,
    max_len=max_len,
).to(device)

criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=1e-4,
    betas=(0.9, 0.98),
    eps=1e-9,
)

# ===============================
# 5. Training loop
# ===============================

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for batch in train_dataloader:
        src = batch["src"].to(device)
        tgt = batch["tgt"].to(device)

        # Teacher forcing: decoder input is tgt[:, :-1], labels are tgt[:, 1:]
        tgt_inp = tgt[:, :-1]
        tgt_out = tgt[:, 1:]

        logits = model(src, tgt_inp, pad_idx, pad_idx)  # [batch, tgt_len-1, vocab]

        loss = criterion(
            logits.reshape(-1, tgt_vocab_size),
            tgt_out.reshape(-1),
        )

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}: loss = {avg_loss:.4f}")
